In [4]:
import pandas as pd
import json
from pymongo import MongoClient
import hashlib

In [5]:
def review_to_df(file_path, k):
    """read and process the first k rows of review data

    Args:
        file_path (string): path to .json file
        k (int): first k lines to convert
    Returns:
        DataFrame: transformed dataframe
    """
    keys_all_set = set()

    # find all feature names
    with open(file_path, 'r') as json_file:
        line_count = 0
        for line in json_file:
            if line_count < k:
                data = json.loads(line)
                keys_all_set.update(data.keys())
            else:
                break
            line_count += 1
    
    # read first k rows 
    data_dict = {key: [] for key in keys_all_set}

    with open(file_path, 'r') as json_file:
        line_count = 0
        for line in json_file:
            if line_count < k:
                data = json.loads(line)
                if "verified" in data and data["verified"] == True:
                    for key, value in data.items():
                        data_dict[key].append(value)
                    keys_per_line = set(data.keys()) 
                    keys_difference = keys_all_set - keys_per_line 
                    for key in keys_difference:
                        data_dict[key].append(None)
            else:
                break
            line_count += 1

    # for key, value in data_dict.items():
    #     print(key)
    #     print(len(value))
    df = pd.DataFrame(data_dict)
    print(f"The dataframe has {df.shape[0]} rows of unqiue review")
    return df

In [6]:
df = review_to_df("AMAZON_FASHION.json", 800000)

The dataframe has 750950 rows of unqiue review


In [7]:
def metadata_to_csv(file_path, products_to_find):
    """find and process products info to dataframe

    Args:
        file_path (string): path to file
        products_to_find (numpy): unique product ID
    Returns:
        DataFrame: transformed dataframe
    """
    keys_all_set = set()

    # find all feature names
    with open(file_path, 'r') as json_file:
        for line in json_file:
            data = json.loads(line)
            keys_all_set.update(data.keys())

    # find desired products' info
    desired_set = set(products_to_find)
    data_dict = {key: [] for key in keys_all_set}
    with open(file_path, 'r') as json_file:
        for line in json_file:
            data = json.loads(line)
            product_id = data.get("asin")
            if product_id in desired_set:
                for key, value in data.items():
                    data_dict[key].append(value)
                keys_per_line = set(data.keys()) 
                keys_difference = keys_all_set - keys_per_line 
                for key in keys_difference:
                    data_dict[key].append(None)
    
    df = pd.DataFrame(data_dict)
    print(f"The dataframe has {df.shape[0]} rows of unqiue products' information")
    return df

In [8]:
raw_dataframe = metadata_to_csv("meta_AMAZON_FASHION.json", df['asin'].unique())
# print(raw_dataframe.shape)
raw_dataframe = raw_dataframe.drop_duplicates(subset='asin')
# print(raw_dataframe.shape)
# clean the duplicated asin in the raw_dataframe
# raw_dataframe = metadata_to_csv("meta_AMAZON_FASHION.json").drop_duplicates(subset=['asin'])

The dataframe has 147413 rows of unqiue products' information


In [9]:
final_dataset = raw_dataframe[['asin','title','rank','brand','imageURL','feature','price']]
final_dataset

,asin,title,rank,brand,imageURL,feature,price
0,0764443682,Slime Time Fall Fest [With CDROM and Collector...,"13,052,976inClothing,Shoesamp;Jewelry(",Group Publishing (CO),[https://images-na.ssl-images-amazon.com/image...,[Product Dimensions:\n \n8....,None
1,1291691480,XCC Qi promise new spider snake preparing men'...,"11,654,581inClothing,Shoesamp;Jewelry(",None,[https://images-na.ssl-images-amazon.com/image...,None,None
2,1940735033,"Ashes to Ashes, Oranges to Oranges","19,734,184inClothing,ShoesJewelry(",Flickerlamp Publishing,[https://images-na.ssl-images-amazon.com/image...,[Package Dimensions:\n \n8....,None
3,1942705034,"365 Affirmations for a Year of Love, Peace & P...","16,179,013inClothing,Shoesamp;Jewelry(",None,[https://images-na.ssl-images-amazon.com/image...,None,None
4,3293015344,Blessed by Pope Benedetto XVI Wood Religious B...,"7,787,039inClothing,ShoesJewelry(",None,None,[Package Dimensions:\n \n4....,None
...,...,...,...,...,...,...,...
147408,B01HI7K476,"O.RIYA I'd Pick You Every Time Guitar Pick, Mu...","87,113inClothing,ShoesJewelry(",O.RIYA,[https://images-na.ssl-images-amazon.com/image...,[Gift For Him or Daddy -The perfect gift for H...,$9.69
147409,B01HI9YG0U,Fashion Women Stainless Steel Watch Ladies Cas...,"1,224,687inClothing,ShoesJewelry(",None,[https://images-na.ssl-images-amazon.com/image...,[Package Dimensions:\n \n4....,None
147410,B01HIVIU82,Singbring Women's Outdoor Lightweight Waterpro...,"1,910,835inClothing,ShoesJewelry(",Singbring,None,[Package Dimensions:\n \n13...,None
147411,B01HJEOC9E,ROMWE Women's Bohemian Short Sleeve V neck Lon...,"1,736,074inClothing,ShoesJewelry(",None,[https://images-na.ssl-images-amazon.com/image...,[Package Dimensions:\n \n11...,None


In [10]:
non_none_values = final_dataset[final_dataset.notna().all(axis=1)]

pattern = r'(\d{1,3}(?:,\d{3})*)(?=in)'

# Extract the numbers with commas from the 'rank' column using the regex pattern
non_none_values['extracted_rank'] = non_none_values['rank'].str.extract(pattern)

# Remove the commas to convert the string into a pure number string
non_none_values['extracted_rank'] = non_none_values['extracted_rank'].str.replace(',', '')

# Convert the cleaned-up 'extracted_rank' column to numeric (integers)
non_none_values['extracted_rank'] = pd.to_numeric(non_none_values['extracted_rank'], errors='coerce')

# # Display the DataFrame to verify the changes
# print(non_none_values[['rank', 'extracted_rank']])

# drop the original 'rank' column
non_none_values = non_none_values.drop(columns=['rank'])

# rename the 'extracted_rank' column to 'rank'
non_none_values = non_none_values.rename(columns={'extracted_rank': 'rank'})

/var/folders/w7/zlh4zd_j1jxc2nn97w4h0t700000gn/T/ipykernel_37865/2741160071.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_none_values['extracted_rank'] = non_none_values['rank'].str.extract(pattern)
/var/folders/w7/zlh4zd_j1jxc2nn97w4h0t700000gn/T/ipykernel_37865/2741160071.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_none_values['extracted_rank'] = non_none_values['extracted_rank'].str.replace(',', '')
/var/folders/w7/zlh4zd_j1jxc2nn97w4h0t700000gn/T/ipykernel_37865/2741160071.py:12: S

In [11]:
def simplify_strings(lst): 
    return [item.replace('\n                    \n', ' ') for item in lst]
non_none_values['feature'] = non_none_values['feature'].apply(simplify_strings)

# change the column name asin to _id
non_none_values = non_none_values.rename(columns={'asin': '_id'})

# store the dataframe to a json file
non_none_values.to_json('final_dataset.json', orient='records', lines=True)
                        


In [12]:
non_none_values
non_none_values['price'] = non_none_values['price'].str.replace('$', '')
non_none_values['price'] = non_none_values['price'].str.replace('$', '').str.replace(',', '').str.split('-').str[0].astype(float)


In [13]:
non_none_values = non_none_values[~non_none_values['title'].str.contains("aPageStart")]

In [14]:
non_none_values

,_id,title,brand,imageURL,feature,price,rank
13,9654263246,"X. L. Carbon Fiber Money Clip, made in the USA",Roar Carbon,[https://images-na.ssl-images-amazon.com/image...,"[Real Carbon Fiber, Made in USA, 5 year warran...",14.99,3725957.0
14,B00004T3SN,Shimmer Anne Shine Clip On Costume/Halloween C...,Shimmer Anne Shine,[https://images-na.ssl-images-amazon.com/image...,[Shimmer Anne Shine Clip On Costume/Halloween ...,6.99,468314.0
36,B00007GDFV,Buxton Heiress Pik-Me-Up Framed Case,Buxton,[https://images-na.ssl-images-amazon.com/image...,"[Leather, Imported, synthetic lining, Flap clo...",16.95,43930.0
206,B00023JX9Y,Art Nouveau Sterling Silver Ornate Repousse He...,Silver Insanity,[https://images-na.ssl-images-amazon.com/image...,"[2&5/8"" High and 3/4"" Wide, Weight is Approx. ...",44.66,6343439.0
241,B0002PR25Y,Silver Forest Surgical Steel Turquoise Filigre...,Silver Forest,[https://images-na.ssl-images-amazon.com/image...,[Shipping Weight: 1.28 ounces (View shipping r...,23.00,2852593.0
...,...,...,...,...,...,...,...
147397,B01HH37KTG,Opal Gem Clip On Nose Ring Fake Non No Piercin...,Pierced Owl,[https://images-na.ssl-images-amazon.com/image...,"[Opal Gem Non Piercing Nose Clip, Silver Tone ...",7.99,134340.0
147406,B01HHVC958,Coxeer Venetian Masquerade Mask Halloween Mard...,Coxeer,[https://images-na.ssl-images-amazon.com/image...,[The venetian mask is made of high-quality and...,12.98,67372.0
147407,B01HI7FZLQ,"O.RIYA Gifts I Pick You Always Forever, Father...",O.RIYA,[https://images-na.ssl-images-amazon.com/image...,[Tell him he's special with this stainless ste...,11.96,540091.0
147408,B01HI7K476,"O.RIYA I'd Pick You Every Time Guitar Pick, Mu...",O.RIYA,[https://images-na.ssl-images-amazon.com/image...,[Gift For Him or Daddy -The perfect gift for H...,9.69,87113.0


In [15]:
import hashlib

def complex_hash(asin, num_shards=3):
    # Use SHA-256 hash function to hash the ASIN
    hash_object = hashlib.sha256(asin.encode())
    # Get the hexadecimal representation of the hash
    hex_dig = hash_object.hexdigest()
    # Convert the hex digest to an integer
    int_hash = int(hex_dig, 16)
    # Use the modulo operation to get an index for the shard
    shard_index = int_hash % num_shards
    return shard_index



In [16]:
from pymongo import MongoClient

# Connect to the MongoDB client
client = MongoClient('mongodb://localhost:27017/')

# collection name list
db_list = ['db_1', 'db_2', 'db_3']

# Create a new database and collection
db_backup = client['db_backup']

# Create a new collection
collection_backup = db_backup['collection_backup']

# Assuming 'non_none_values' is your DataFrame
for index, row in non_none_values.iterrows():
    # Convert the row to a dictionary
    num = complex_hash(row['_id'])
    db = client[db_list[num]]
    collection = db['collection']
    row_dict = row.to_dict()
    # Insert the document into MongoDB
    collection.insert_one(row_dict)
    collection_backup.insert_one(row_dict)
